## First Demo - Adhesion Demo

```dim_x = dim_y = 100
specs = [
    Metadata(num_processors=4),
    PottsCore(dim_x=dim_x,
              dim_y=dim_y,
              neighbor_order=2,
              boundary_x="Periodic",
              boundary_y="Periodic")
]

cell_types = ["T1", "T2", "T3"]

for ct in cell_types:
    volume_specs.param_new(ct, target_volume=25, lambda_volume=2)

contact_specs = ContactPlugin(neighbor_order=2)
for idx1 in range(len(cell_types)):
    contact_specs.param_new(type_1="Medium", type_2=cell_types[idx1], energy=16)
    for idx2 in range(idx1, len(cell_types)):
        contact_specs.param_new(type_1=cell_types[idx1], type_2=cell_types[idx2], energy=16)

adhesion_specs = AdhesionFlexPlugin(neighbor_order=2)
adhesion_specs.density_new(molecule="M1", cell_type="T1", density=1.0)
adhesion_specs.density_new(molecule="M2", cell_type="T2", density=1.0)
formula = adhesion_specs.formula_new(formula_name='Binary')
formula.param_set("M1", "M1", -10.0)
formula.param_set("M1", "M2", 0.0)
formula.param_set("M2", "M2", 10.0)

from cc3d.core.PyCoreSpecs import UniformInitializer

unif_init_specs = UniformInitializer()
unif_init_specs.region_new(width=5, pt_min=(0, 0, 0), pt_max=(dim_x, dim_y, 1),
                           cell_types=["T1", "T1", "T2", "T2", "T3"])
```

In [2]:
from Spec_UI_1 import SpecificationSetupUI

# ui = SpecificationSetupUI()

# Debuging: 
# + Random seed should be a number generator number (positive integer)
# + Reset celltype and reset pottscore needs to be fixed? 
# Neighbor order can't be negative for plugins 

# Newly added features:
# Dimension were bounded now. 
# Cell Type id added. (smallest available id) 
# Advanced potts core setting added. 

# Stretch goal: 
# Try the Sorting Notebook demo, start looking at energy and initializers.


In [3]:
from Spec_UI import SpecificationSetupUI

ui = SpecificationSetupUI()

In [4]:
# Testing vertical tabs 
import ipywidgets as widgets
from IPython.display import display

# Create content for each tab
out1 = widgets.Output()
out2 = widgets.Output()
out3 = widgets.Output()

with out1:
    print("Content of Tab 1")

with out2:
    print("Content of Tab 2")

with out3:
    print("Content of Tab 3")

# Create individual buttons instead of ToggleButtons for more control
tab1_btn = widgets.Button(description='Tab 1', layout=widgets.Layout(width='100%'))
tab2_btn = widgets.Button(description='Tab 2', layout=widgets.Layout(width='100%'))
tab3_btn = widgets.Button(description='Tab 3', layout=widgets.Layout(width='100%'))

# Put them into a VBox so they stack tightly
tabs_box = widgets.VBox([tab1_btn, tab2_btn, tab3_btn], layout=widgets.Layout(width='120px'))

# Output area
content_area = widgets.Output()

# Function to switch content
def show_tab(tab):
    content_area.clear_output()
    with content_area:
        if tab == 1:
            display(out1)
        elif tab == 2:
            display(out2)
        elif tab == 3:
            display(out3)

# Attach callbacks
tab1_btn.on_click(lambda b: show_tab(1))
tab2_btn.on_click(lambda b: show_tab(2))
tab3_btn.on_click(lambda b: show_tab(3))

# Layout the tabs and content
ui = widgets.HBox([tabs_box, content_area])

# Display
display(ui)

# Initialize first tab
show_tab(1)


In [5]:
import ipywidgets as widgets
from IPython.display import display
from cc3d.core.PyCoreSpecs import PLUGINS

for plugin_cls in PLUGINS:
    plugin = plugin_cls()
    print(f"Plugin: {plugin_cls.__name__}")
    for param_name in plugin.spec_dict.keys():
        # For demonstration, create a text box for each parameter
        value = plugin.spec_dict[param_name]
        input_widget = widgets.Text(
            value=str(value),
            description=param_name
        )
        display(input_widget)

Plugin: AdhesionFlexPlugin


Text(value='1', description='neighbor_order')

Text(value='[]', description='molecules')

Text(value='{}', description='densities')

Text(value='{}', description='binding_formulas')

...

Text(value='{}', description='field_specs')

Plugin: SurfacePlugin


Text(value='{}', description='params')

Plugin: VolumePlugin


Text(value='{}', description='params')

In [6]:
from cc3d.core.PyCoreSpecs import PLUGINS
print([p.__name__ for p in PLUGINS])

['AdhesionFlexPlugin', 'BoundaryPixelTrackerPlugin', 'CellTypePlugin', 'CenterOfMassPlugin', 'ChemotaxisPlugin', 'ConnectivityGlobalPlugin', 'ConnectivityPlugin', 'ContactPlugin', 'CurvaturePlugin', 'ExternalPotentialPlugin', 'FocalPointPlasticityPlugin', 'LengthConstraintPlugin', 'MomentOfInertiaPlugin', 'NeighborTrackerPlugin', 'PixelTrackerPlugin', 'SecretionPlugin', 'SurfacePlugin', 'VolumePlugin']
